# Operator Upgrade Tests

## Setup Seldon Core

Follow the instructions to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core).

In [1]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [2]:
import json
import time

In [5]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [6]:
VERSION=!cat ../version.txt
VERSION=VERSION[0]
VERSION

'1.19.0-dev'

## Install Stable Version

In [3]:
!kubectl create namespace seldon-system

Error from server (AlreadyExists): namespaces "seldon-system" already exists


In [4]:
!helm upgrade seldon seldon-core-operator --repo https://storage.googleapis.com/seldon-charts --namespace seldon-system --set istio.enabled=true --wait

Release "seldon" has been upgraded. Happy Helming!
NAME: seldon
LAST DEPLOYED: Thu Dec  4 11:51:27 2025
NAMESPACE: seldon-system
STATUS: deployed
REVISION: 2
TEST SUITE: None


## Launch a Range of Models

In [7]:
%%writetemplate resources/model.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier:{VERSION}
          name: classifier
    graph:
      name: classifier
      type: MODEL
      endpoint:
        type: REST
    name: example
    replicas: 1

In [8]:
!kubectl create -f resources/model.yaml -n seldon

seldondeployment.machinelearning.seldon.io/seldon-model created


In [9]:
%%writefile ../servers/tfserving/samples/halfplustwo_rest.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: hpt
spec:
  name: hpt
  protocol: tensorflow
  transport: rest
  predictors:
  - graph:
      name:  halfplustwo
      implementation: TENSORFLOW_SERVER
      modelUri: gs://seldon-models/tfserving/half_plus_two
      parameters:
        - name: model_name
          type: STRING
          value: halfplustwo
    name: default
    replicas: 1

Overwriting ../servers/tfserving/samples/halfplustwo_rest.yaml


In [10]:
!kubectl create -f ../servers/tfserving/samples/halfplustwo_rest.yaml -n seldon

seldondeployment.machinelearning.seldon.io/hpt created


In [11]:
%%writefile ../examples/models/payload_logging/model_logger.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: model-logs
spec:
  name: model-logs
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier_rest:1.3
          name: classifier
          imagePullPolicy: Always
    graph:
      name: classifier
      type: MODEL
      endpoint:
        type: REST
      logger:
        url: http://logger.seldon/
        mode: all
    name: logging
    replicas: 1

Writing ../examples/models/payload_logging/model_logger.yaml


In [12]:
!kubectl create -f ../examples/models/payload_logging/model_logger.yaml -n seldon

seldondeployment.machinelearning.seldon.io/model-logs created


Wait for all models to be available

In [14]:
!kubectl wait sdep/hpt --for=condition=ready --timeout=120s -n seldon
!kubectl wait sdep/model-logs --for=condition=ready --timeout=120s -n seldon
!kubectl wait sdep/seldon-model --for=condition=ready --timeout=120s -n seldon

seldondeployment.machinelearning.seldon.io/hpt condition met
seldondeployment.machinelearning.seldon.io/model-logs condition met
seldondeployment.machinelearning.seldon.io/seldon-model condition met


## Count the number of resources

In [16]:
def getOwned(raw):
    count = 0
    for res in raw["items"]:
        if (
            "ownerReferences" in res["metadata"]
            and res["metadata"]["ownerReferences"][0]["kind"] == "SeldonDeployment"
        ):
            count += 1
    return count


def getResourceStats():
    # Get number of deployments
    dps = !kubectl get deployment -n seldon -o json
    dps = json.loads("".join(dps))
    numDps = getOwned(dps)
    print("Number of deployments owned", numDps)

    # Get number of services
    svcs = !kubectl get svc -n seldon -o json
    svcs = json.loads("".join(svcs))
    numSvcs = getOwned(svcs)
    print("Number of services owned", numSvcs)

    # Get number of virtual services
    vss = !kubectl get vs -n seldon -o json
    vss = json.loads("".join(vss))
    numVs = getOwned(vss)
    print("Number of virtual services owned", numVs)

    # Get number of hpas
    hpas = !kubectl get hpa -o json
    hpas = json.loads("".join(hpas))
    numHpas = getOwned(hpas)
    print("Number of hpas owned", numHpas)

    return (numDps, numSvcs, numVs, numHpas)


(dp1, svc1, vs1, hpa1) = getResourceStats()

Number of deployments owned 3
Number of services owned 6
Number of virtual services owned 3
Number of hpas owned 0


## Upgrade to latest

In [17]:
!helm upgrade seldon ../helm-charts/seldon-core-operator --namespace seldon-system --set istio.enabled=true --wait

Release "seldon" has been upgraded. Happy Helming!
NAME: seldon
LAST DEPLOYED: Thu Dec  4 11:54:40 2025
NAMESPACE: seldon-system
STATUS: deployed
REVISION: 3
TEST SUITE: None


In [19]:
!kubectl wait sdep/hpt --for=condition=ready --timeout=120s -n seldon
!kubectl wait sdep/model-logs --for=condition=ready --timeout=120s -n seldon
!kubectl wait sdep/seldon-model --for=condition=ready --timeout=120s -n seldon

seldondeployment.machinelearning.seldon.io/hpt condition met
seldondeployment.machinelearning.seldon.io/model-logs condition met
seldondeployment.machinelearning.seldon.io/seldon-model condition met


In [20]:
# Give time for resources to terminate
for i in range(120):
    (dp2, svc2, vs2, hpa2) = getResourceStats()
    if dp1 == dp2 and svc1 == svc2 and vs1 == vs2 and hpa1 == hpa2:
        break
    time.sleep(1)
assert dp1 == dp2
assert svc1 == svc2
assert vs1 == vs2
assert hpa1 == hpa2

Number of deployments owned 3
Number of services owned 6
Number of virtual services owned 3
Number of hpas owned 0


In [21]:
!kubectl delete -f resources/model.yaml -n seldon
!kubectl delete -f ../examples/models/payload_logging/model_logger.yaml -n seldon
!kubectl delete -f ../servers/tfserving/samples/halfplustwo_rest.yaml -n seldon

seldondeployment.machinelearning.seldon.io "seldon-model" deleted
seldondeployment.machinelearning.seldon.io "model-logs" deleted
seldondeployment.machinelearning.seldon.io "hpt" deleted
